In [1]:
import os
import time
import pandas as pd

#in
main = os.getcwd()
TopicWithLabel = main + "/../../result/modelOutput/TopicWithLabel.xlsx"
TopicWithValue = main + "/../../result/modelOutput/TopicWithValue.csv"
masterFile  = main + "/../../data/ModelingDataset/V.2/MasterDesctiption.csv" 
#test 
masterFileasd  = main + "/../../data/ModelingDataset/V.2/MasterDesctiptionTestasddasd.csv" 

#out
output_reuslt = main + "/../../result/integrationOutput/"

#### Extract Label Data

In [ ]:
import pandas as pd

# Load the Excel file
df = pd.read_excel(TopicWithLabel)
# List of industries
industries = ['AGRO', 'CONSUMP', 'FINCIAL', 'INDUS', 'PROPCON', 'RESOURC', 'SERVICE', 'TECH']

# Dictionary to hold topics for each industry
industry_topics = {industry: [] for industry in industries}

# Loop through each industry and extract topics
for industry in industries:
    industry_rows = df[df['Industry'].str.contains(industry, na=False)]
    topics = industry_rows['Topic'].tolist()
    industry_topics[industry] = topics

# industry_topics now contains the list of topics for each industry


#### Select Top 3 

In [ ]:
import pandas as pd

# Load your Excel file
df = pd.read_csv(TopicWithValue)

# Creating a reverse mapping from topics to industries
topic_to_industry = {}
for industry, topics in industry_topics.items():
    for topic in topics:
        topic_number = topic.split()[1]  # Extract the topic number
        topic_to_industry[f'Topic_{topic_number}'] = industry

# Convert topic columns to numeric, coerce errors to NaN
topic_columns = [col for col in df.columns if col.startswith('Topic_')]
for col in topic_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Prepare a list to store the Article_ID with their top 3 industries
articles_with_top_industries = []

# Iterate over the DataFrame rows
for index, row in df.iterrows():
    article_id = row['Article_ID']
    # Get the top 3 values and their indices from the topic columns
    top_values = row[topic_columns].dropna().nlargest(3)
    top_indices = top_values.index

    # Translate the top topic indices to their respective industries
    top_industries = [topic_to_industry.get(topic) for topic in top_indices]

    # Append the result to the list
    articles_with_top_industries.append({'Article_ID': article_id, 'Tags': ', '.join(filter(None, top_industries))})

# Create a DataFrame from the list
articles_df = pd.DataFrame(articles_with_top_industries)

# Save the DataFrame to a CSV file
output_Topic_Industries = output_reuslt + 'Topic_Industries.csv'
articles_df.to_csv(output_Topic_Industries, index=False)


In [ ]:
output_Topic_Industries

#### Normalize

In [ ]:
import pandas as pd

# Merging
df1 = pd.read_csv(output_Topic_Industries) #top 3
df2 = pd.read_csv(masterFileasd) # master

assert len(df1) == len(df2)

for column in df2.columns:
    df1[column] = df2[column].values

# Consoildate
df1.drop(['Article_ID', 'Status'], axis=1, inplace=True)

df1['Tags'] = df1['Tags'].apply(lambda x: x.split(",")[0].strip('"'))

df1.rename(columns={'Tags': 'News_Industry'}, inplace=True)

df1 = df1.reindex(columns=['ID', 'Agency', 'Date', 'Title', 'Links', 'News_Industry'])

df1['Date'] = pd.to_datetime(df1['Date'])

# Normalize the 'Date' column to retain only the date part
df1['Date'] = df1['Date'].dt.date

output_Topic_Industries_merged = output_reuslt + 'Topic_Industries_Merged.csv'
df1.to_csv(output_Topic_Industries_merged, index=False)




#### Final

In [ ]:
final_df = df1.copy()

# Convert the 'Date' column to datetime
final_df['Date'] = pd.to_datetime(final_df['Date'])

final_df = final_df[final_df['Date'].dt.year == 2023]

# Remove duplicate rows based on 'ID'
final_df = final_df.drop_duplicates(subset='ID')

# Sort the DataFrame by the 'Date' column
final_df = final_df.sort_values(by='Date')
final_df.drop(['ID'], axis=1, inplace=True)

ArticleWithTags = output_reuslt + 'ArticleWithTags.csv'
final_df.to_csv(ArticleWithTags, index=False)

In [ ]:
len(final_df)

#### Result decrease version

In [2]:
ArticleWithTags = output_reuslt + 'ArticleWithTags.csv'

df = pd.read_csv(ArticleWithTags)
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
mask = (df['Date'] > '2023-1-1') & (df['Date'] <= '2023-1-1')

In [ ]:
df

In [3]:
import pandas as pd

# Assuming df is your DataFrame

# Make sure 'Date' is in datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Define a custom function to apply the limit
def limit_articles_per_group(group):
    return group.head(15)

# Group by 'Date' and 'news_industry', then apply the limit
limited_df = df.groupby(['Date', 'News_Industry'], group_keys=False).apply(limit_articles_per_group)

# Assuming the DataFrame is now significantly reduced, you might want to sort and then take the first 40,000 rows
# This step is optional and assumes you want to strictly limit the output size
# limited_df = limited_df.sort_values(by=['Date', 'news_industry']).head(40000)

# Save the result to a new CSV file
limitedArticles = output_reuslt + 'limited_articles.csv'
limited_df.to_csv(limitedArticles, index=False)

print(f"Total rows after applying limits: {len(limited_df)}")


Total rows after applying limits: 32547


In [4]:
limited_df

,Agency,Date,Title,Links,News_Industry
19,thairath,2023-01-01,“คิมจองอึน” สั่งเร่งพัฒนาขีปนาวุธข้ามทวีป-ขยาย...,https://www.thairath.co.th/news/local/2591993,AGRO
40,thairath,2023-01-01,‡∏ä‡∏≤‡∏ß‡πÄ‡∏ô‡πá‡∏ï‡∏Ç‡∏∏‡∏î‡∏Ñ‡∏≥‡∏û‡∏π‡∏î ...,https://www.thairath.co.th/news/local/2591456,AGRO
90,thairath,2023-01-01,"เอ๊ะยังไง เสื้อแข่งปักชื่อ ""เมสซี"" โผล่ในช็อปท...",https://www.thairath.co.th/news/local/2592336,AGRO
96,thairath,2023-01-01,ออร่าประกาย “เนวิน” ชูเหตุผลสำคัญ ทำมั่นใจ “ศุ...,https://www.thairath.co.th/news/local/2592311,AGRO
133,thairath,2023-01-01,ฮอนดะติวเข้ม เช็กสถิติ 6 ครั้ง ทีมชาติไทย ดวล ...,https://www.thairath.co.th/news/local/2592090,AGRO
...,...,...,...,...,...
147510,thairath,2023-12-31,"ไม่ดีพอ ""กูรูดัง"" รุมจวก ""3 แข้งแมนยูฯ"" เกมบุก...",https://www.thairath.co.th/news/local/2751734,SERVICE
147512,thairath,2023-12-31,"แย่กว่านี้มีอีกไหม เปิด 6 ความห่วย ""แมนยูฯ"" ยุ...",https://www.thairath.co.th/news/local/2751736,SERVICE
147515,thairath,2023-12-31,สภาพอากาศวันนี้ ภาคเหนือยังหนาว คนไทยแห่เที่ยว...,https://www.thairath.co.th/news/local/2751741,SERVICE
147637,thairath,2023-12-31,ช่อง 7 เปิดโผละครใหม่ ส่งท้ายปีเก่า ต้อนรับปี ...,https://www.thairath.co.th/news/local/2751649,TECH


#### test

In [5]:
import pandas as pd

# Assuming df is your DataFrame with the correct 'Date' and 'news_industry' columns

def print_news_count_by_industry_and_day(df):
    # Group by 'Date' and 'news_industry' and count the number of articles
    counts = df.groupby(['Date', 'News_Industry']).size().reset_index(name='Article Count')
    
    # Sort the results for better readability (optional)
    counts = counts.sort_values(by=['Date', 'News_Industry'])
    
    # Print the results
    for index, row in counts.iterrows():
        print(f"Date: {row['Date'].strftime('%Y-%m-%d')}, Industry: {row['News_Industry']}, Articles: {row['Article Count']}")

# Example usage of the function
print_news_count_by_industry_and_day(df)


Date: 2023-01-01, Industry: AGRO, Articles: 8
Date: 2023-01-01, Industry: FINCIAL, Articles: 48
Date: 2023-01-01, Industry: INDUS, Articles: 8
Date: 2023-01-01, Industry: PROPCON, Articles: 5
Date: 2023-01-01, Industry: RESOURC, Articles: 35
Date: 2023-01-01, Industry: SERVICE, Articles: 102
Date: 2023-01-01, Industry: TECH, Articles: 1
Date: 2023-01-02, Industry: AGRO, Articles: 10
Date: 2023-01-02, Industry: FINCIAL, Articles: 42
Date: 2023-01-02, Industry: INDUS, Articles: 11
Date: 2023-01-02, Industry: PROPCON, Articles: 6
Date: 2023-01-02, Industry: RESOURC, Articles: 45
Date: 2023-01-02, Industry: SERVICE, Articles: 116
Date: 2023-01-02, Industry: TECH, Articles: 4
Date: 2023-01-03, Industry: AGRO, Articles: 14
Date: 2023-01-03, Industry: FINCIAL, Articles: 131
Date: 2023-01-03, Industry: INDUS, Articles: 10
Date: 2023-01-03, Industry: PROPCON, Articles: 6
Date: 2023-01-03, Industry: RESOURC, Articles: 91
Date: 2023-01-03, Industry: SERVICE, Articles: 157
Date: 2023-01-03, Indust

In [10]:
df = pd.read_csv(limitedArticles)
df['Date'] = pd.to_datetime(df['Date'])
maskedDate = df['Date'] == '2023-01-01'

len(df.loc[maskedDate])


67

In [7]:
import pandas as pd

# Example setup (replace this with your actual DataFrame)
data = {
    'Date': ['2023-01-01'] * 7,  # Simulating entries for a single day
    'News_Industry': ['AGRO', 'FINCIAL', 'INDUS', 'PROPCON', 'RESOURC', 'SERVICE', 'TECH'],
    'Articles': [8, 48, 8, 5, 35, 102, 1]  # The counts (for illustration only)
}
df = pd.DataFrame(data)
df['Date'] = pd.to_datetime(df['Date'])


def limit_articles(df):
    # Assuming 'Date' is already in the correct format
    # Limit articles to 15 per industry per day
    df['Rank'] = df.groupby(['Date', 'News_Industry']).cumcount() + 1
    limited_df = df[df['Rank'] <= 15]

    return limited_df.drop('Rank', axis=1)

# Assuming df is your DataFrame after initial processing but before applying the 15 article limit
limited_df = limit_articles(df)

# Now, print the count using the print function described earlier
print_news_count_by_industry_and_day(limited_df)


Date: 2023-01-01, Industry: AGRO, Articles: 1
Date: 2023-01-01, Industry: FINCIAL, Articles: 1
Date: 2023-01-01, Industry: INDUS, Articles: 1
Date: 2023-01-01, Industry: PROPCON, Articles: 1
Date: 2023-01-01, Industry: RESOURC, Articles: 1
Date: 2023-01-01, Industry: SERVICE, Articles: 1
Date: 2023-01-01, Industry: TECH, Articles: 1


In [8]:
df

,Date,News_Industry,Articles,Rank
0,2023-01-01,AGRO,8,1
1,2023-01-01,FINCIAL,48,1
2,2023-01-01,INDUS,8,1
3,2023-01-01,PROPCON,5,1
4,2023-01-01,RESOURC,35,1
5,2023-01-01,SERVICE,102,1
6,2023-01-01,TECH,1,1


In [10]:
ArticleWithTags ='E:/sp2023stock/TopicModeling/result/integrationOutput/Oat/ArticleWithTags.csv'
df =  pd.read_csv(ArticleWithTags)
df.drop(['Date','Title','News_Industry'], axis=1, inplace=True)
df.to_csv('E:/sp2023stock/TopicModeling/result/integrationOutput/Oat/ArticleWithTagsTest.csv', index= False)